#### ⚙️ Importações

In [1]:
# !pip install pycaret
# !pip install Unidecode

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
# from google.colab import drive

# drive.mount('/content/drive')
# df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/censo_cultural.xlsx')

In [4]:
# df = pd.read_excel('/mnt/g/Meu Drive/Colab Notebooks/censo_cultural.xlsx')
df = pd.read_excel('../data/censo_cultural.xlsx')

#### 🔌 Functions

##### Renomear colunas

In [5]:
def rename_columns() -> None:
    df.rename(
        columns={
            "2) Qual o seu curso?": "curso",
            "3) Qual o Ano que está cursando? ": "serie",
            "5) Você é um aluno(a)...  ": "tipo_residente",
            "6) Quais dessas ATIVIDADES CULTURAIS você participa ou já participou?": "atv_culturais",
            "7) Quais dessas HABILIDADES MUSICAIS você possui?": "hab_musicais",
            "8) Quais desses ESPORTES você pratica?": "esportes",
            "9) Quais desses JOGOS você pratica?": "jogos",
            "10) Quais dessas LUTAS você pratica?": "lutas",
            "11) Em relação às 5 perguntas anteriores, quais dessas atividades você se considera bom e capaz de ensinar aos seus colegas?": "melhores_atividades",  # remover
            "12) Quais dessas ATIVIDADES CULTURAIS você gostaria de participar?": "atv_culturais_a_fazer",
            "13) Quais dessas HABILIDADES MUSICAIS você gostaria de possuir?": "hab_musicais_desejadas",
            "14) Quais desses ESPORTES você gostaria de praticar?": "esportes_desejados",
            "15) Quais desses JOGOS você gostaria de praticar?": "jogos_desejados",
            "16) Quais dessas LUTAS você gostaria de praticar?": "lutas_desejadas",
            "17) Em quais HORÁRIOS você teria disponibilidades para exercer Atividades Culturais?": "horarios_disponiveis",  # remover
            "18) Em quais LOCAIS você teria disponibilidades para exercer Atividades Culturais ": "locais_disponiveis",  # remover
            "19) Em relação às ESTRUTURAS EXISTENTES, o que você gostaria que melhorasse em nossa instituição?": "melhoria_estrutura",
            "20) Em relação às NOVAS ESTRUTURAS, o que você desejaria que houvesse em nossa instituição?": "novas_estruturas",
            "21) Em relação à ALIMENTAÇÃO, o que você desejaria que fosse melhorado em nossa instituição?": "melhoria_alimentacao",
            "22) Em relação ao TRANSPORTE, o que você desejaria que fosse melhorado?": "transporte",
            "23) Quais desse PROJETOS ESPECIAIS, você teria interesse em participar?": "projetos_a_participar",
        },
        inplace=True,
    )

##### Tratamento de strings

In [6]:
def encoder(series_names: list)->None:
   for serie_name in series_names:
      enc = LabelEncoder()
      df[serie_name] = enc.fit_transform(df[serie_name])

In [7]:
import unidecode as uni

def remove_accent(name):
    return uni.unidecode(str(name))

##### Dummies

In [8]:
def insert_dummies(df: pd.DataFrame, dummies: pd.DataFrame, insert_position: int) -> None:
    df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
    df.drop('aux', axis=1, inplace=True)

#### 📑 Pré-processamento dos dados

In [9]:
rename_columns()

In [10]:
df.drop(
    columns=[
        "Carimbo de data/hora",
        "1) Nome Completo: ",
        "4) Qual a sua Turma? ",
        "melhores_atividades",
        "horarios_disponiveis",
        "locais_disponiveis",
    ],
    inplace=True,
)

In [11]:
for column in df.columns:
    df[column] = df[column].str.lower()
    df[column] = df[column].apply(remove_accent)

In [12]:
encoder([
    'serie',
    'tipo_residente',
])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   curso                   611 non-null    object
 1   serie                   611 non-null    int64 
 2   tipo_residente          611 non-null    int64 
 3   atv_culturais           611 non-null    object
 4   hab_musicais            611 non-null    object
 5   esportes                611 non-null    object
 6   jogos                   611 non-null    object
 7   lutas                   611 non-null    object
 8   atv_culturais_a_fazer   611 non-null    object
 9   hab_musicais_desejadas  611 non-null    object
 10  esportes_desejados      611 non-null    object
 11  jogos_desejados         611 non-null    object
 12  lutas_desejadas         611 non-null    object
 13  melhoria_estrutura      611 non-null    object
 14  novas_estruturas        611 non-null    object
 15  melhor

#### 📑 Pré processamento - Valores unicos

##### atv_culturais

In [14]:
df['aux'] = df['atv_culturais'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('artes marciais', 'luta')
            .replace('capoeira', 'luta')
            .replace('karate', 'luta')
            .replace('canto', 'musica')
            .replace('outro: musica', 'musica')
            .replace('praticava futsal no ginasio da vila sao joao', 'futsal')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('atv_cultural_')

insert_position = df.columns.get_loc('atv_culturais') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### hab_musicais

In [15]:
df['aux'] = df['hab_musicais'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('hab_musical_')

insert_position = df.columns.get_loc('hab_musicais') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### esportes

In [16]:
df['aux'] = df['esportes'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('caminhada/corridaa', 'caminhada/corrida')
            .replace('futbol de campo', 'futebol de campo')
            .replace('futbol de salao', 'futebol de salao')
            .replace('handbol', 'handebol')
            .replace('musculacao', 'academia')
            .replace('karate', 'artes marciais')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('esporte_')

insert_position = df.columns.get_loc('esportes') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### jogos

In [17]:
df['aux'] = df['jogos'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('assim como o de mesa', 'rpg')
            .replace('filho da puta', 'nenhuma opcao')
            .replace('league of legends', 'jogos eletronicos')
            .replace('overwatch', 'jogos eletronicos')
            .replace("(toto)", '')
            .replace('pebolin', 'pebolin/toto')
            .replace('por exemplo o d&d', 'rpg')
            .replace('rpg variados', 'rpg')
            .replace('xadres', 'xadrez')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('jogo_')

insert_position = df.columns.get_loc('jogos') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### lutas

In [18]:
df['aux'] = df['lutas'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('ja pratiquei karate', 'karate')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('luta_')

insert_position = df.columns.get_loc('lutas') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### atv_culturais_a_fazer

In [19]:
df['aux'] = df['atv_culturais_a_fazer'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('aula de bateria', 'aprender algum instrumento')
            .replace('tocar teclado ou outros instrumentos.', 'aprender algum instrumento')
            .replace('(marquei poesia, mas sou boa em leitura, nao em escrever)', '')
            .replace('outras opcoes (especifique) ou observacoes', '')
            .replace('karate', 'luta')
            .replace('jiu-jitsu', 'luta')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('atv_culturais_a_fazer_')

insert_position = df.columns.get_loc('atv_culturais_a_fazer') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### hab_musicais_desejadas

In [20]:
df['aux'] = df['hab_musicais_desejadas'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('percussao', '')
            .replace('percussa', '')
            .replace('bateria/', 'bateria/percussao')
            .replace('violao/guitarra/demais instrumentos de corda', 'canto, instrumentos de corda')
            .replace('canto/violao/guitarra/demais instrumentos de corda', 'canto/canto, instrumentos de corda')
            .replace('canto/canto', 'canto')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('hab_musicais_desejadas_')

insert_position = df.columns.get_loc('hab_musicais_desejadas') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### esportes_desejados

In [21]:
df['aux'] = df['esportes_desejados'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('futbol de campo', 'futebol de campo')
            .replace('futebol de campo/basquete/handebol/pingue pongue/peteca/futmesa', 'futebol de campo, basquete, handebol, pingue pongue, peteca, futmesa')
            .replace('futvilei', 'futvolei')
            .replace('handbol', 'handebol')
            .replace('queimada e atletismo', 'queimada, atletismo')
            .replace('voleiboll', 'voleibol')
            .replace('voleiwheeling e moto crossboll', 'wheeling/moto cross')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('esportes_desejados_')

insert_position = df.columns.get_loc('esportes_desejados') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### jogos_desejados

In [22]:
df['aux'] = df['jogos_desejados'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace("(toto)", '')
            .replace('pebolin', 'pebolin/toto')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('jogos_desejados_')

insert_position = df.columns.get_loc('jogos_desejados') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

##### lutas_desejadas

In [23]:
df['aux'] = df['lutas_desejadas'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace("kung fuh", 'kung fu')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('lutas_desejadas_')

insert_position = df.columns.get_loc('lutas_desejadas') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [24]:
# Analisar viabilidade de usar essa técnica aqui também
# def melt_columns(prefix: str, var_name: str, value_name: str):
#     return df.melt(
#         id_vars=["curso"],
#         value_vars=[col for col in df.columns if col.startswith(prefix)],
#         var_name=var_name,
#         value_name=value_name,
#     )

# atv_melt = melt_columns("atv_cultural_", "atividade", "tipo")


#### 🗃️ Bases - Criar bases para aplicação dos algoritmos

In [25]:
df.drop(columns=[
    'atv_culturais',
    'hab_musicais',
    'esportes',
    'jogos',
    'lutas',
    'atv_culturais_a_fazer',
    'hab_musicais_desejadas',
    'esportes_desejados',
    'jogos_desejados',
    'lutas_desejadas',
], inplace=True)

df.to_csv('../data/df.csv', index=False)

In [26]:
# Buscar os índices de cada coluna para falicitar a divisão das bases de dados
for index, column in enumerate(df.columns):
    print(f"Índice: {index}, Coluna: {column}")

Índice: 0, Coluna: curso
Índice: 1, Coluna: serie
Índice: 2, Coluna: tipo_residente
Índice: 3, Coluna: atv_cultural_apresentacoes musicais
Índice: 4, Coluna: atv_cultural_artesanato
Índice: 5, Coluna: atv_cultural_crio historia e enigmas
Índice: 6, Coluna: atv_cultural_danca
Índice: 7, Coluna: atv_cultural_edicoes de video...
Índice: 8, Coluna: atv_cultural_esporte
Índice: 9, Coluna: atv_cultural_fotografia
Índice: 10, Coluna: atv_cultural_futsal
Índice: 11, Coluna: atv_cultural_grafite/arte urbana
Índice: 12, Coluna: atv_cultural_luta
Índice: 13, Coluna: atv_cultural_musica
Índice: 14, Coluna: atv_cultural_natacao
Índice: 15, Coluna: atv_cultural_nenhuma opcao
Índice: 16, Coluna: atv_cultural_oficinas de gastronomia
Índice: 17, Coluna: atv_cultural_pesquisa cientifica
Índice: 18, Coluna: atv_cultural_piano
Índice: 19, Coluna: atv_cultural_pintura de pano
Índice: 20, Coluna: atv_cultural_pintura/desenho
Índice: 21, Coluna: atv_cultural_poesia/literatura
Índice: 22, Coluna: atv_cultural

##### base das perguntas sobre as atividades que o aluno faz - colunas 0-83

In [27]:
df_atv = df.iloc[:, :83].copy()
df_atv.to_csv('../data/df_atv.csv', index=False)
df_atv

,curso,serie,tipo_residente,atv_cultural_apresentacoes musicais,atv_cultural_artesanato,atv_cultural_crio historia e enigmas,atv_cultural_danca,atv_cultural_edicoes de video...,atv_cultural_esporte,atv_cultural_fotografia,...,luta_boxe,luta_capoeira,luta_jeet kune do,luta_jiu-jitsu,luta_judo,luta_karate,luta_krav maga,luta_kung fu,luta_muay thai,luta_nenhuma opcao
0,agro,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,agro,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,agro,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,agro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,agro,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,mamb,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
607,mamb,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
608,mamb,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
609,mamb,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


##### base das perguntas sobre as atividades culturais que o aluno deseja aprender - colunas 88-157

In [28]:
df_atv_a_fazer = pd.concat([df.iloc[:, :3], df.iloc[:, 83:147]], axis=1)
df_atv_a_fazer.to_csv('../data/df_atv_a_fazer.csv', index=False)
df_atv_a_fazer

,curso,serie,tipo_residente,atv_culturais_a_fazer_aprender algum instrumento,atv_culturais_a_fazer_apresentacoes musicais,atv_culturais_a_fazer_artesanato,atv_culturais_a_fazer_danca,atv_culturais_a_fazer_fotografia,atv_culturais_a_fazer_grafite/arte urbana,atv_culturais_a_fazer_luta,...,jogos_desejados_xadrez,lutas_desejadas_boxe,lutas_desejadas_capoeira,lutas_desejadas_jiu-jitsu,lutas_desejadas_judo,lutas_desejadas_karate,lutas_desejadas_kung fu,lutas_desejadas_muay thai,lutas_desejadas_nenhuma opcao,lutas_desejadas_taekwondo
0,agro,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,agro,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2,agro,0,0,0,0,1,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,agro,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,agro,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,mamb,2,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
607,mamb,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
608,mamb,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
609,mamb,2,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0


##### base das perguntas sobre estrutura - colunas 1-3, 157:

In [29]:
df_estrutura = pd.concat([df.iloc[:, :3], df.iloc[:, 147:]], axis=1)
df_estrutura.to_csv('../data/df_estrutura.csv', index=False)
df_estrutura

,curso,serie,tipo_residente,melhoria_estrutura,novas_estruturas,melhoria_alimentacao,transporte,projetos_a_participar
0,agro,0,1,"recuperacao da sala de jogos, recuperacao do c...","areas para descanso para o intervalo, salas de...",lanches diarios,"criacao de uma ciclovia, para seguranca de que...",recuperacao e ocupacao do antigo cine januaria
1,agro,0,1,"melhora na qualidade e alcance da internet, re...",areas para descanso para o intervalo,lanches diarios,nenhuma opcao,oficina de violao
2,agro,0,0,"melhora na qualidade e alcance da internet, re...","areas para descanso para o intervalo, colchone...","opcoes vegetarianas no almoco, almoco para tod...",melhores condicoes dos onibus terceirizados (m...,recuperacao e ocupacao do antigo cine januaria
3,agro,0,0,"melhora na qualidade e alcance da internet, re...","areas para descanso para o intervalo, salas de...","almoco para todos, lanches diarios, compra de ...","criacao de uma ciclovia, para seguranca de que...",recuperacao e ocupacao do antigo cine januaria
4,agro,0,0,"melhora na qualidade e alcance da internet, su...",salas de aula ao ar livre,"opcoes vegetarianas no almoco, lanches diarios...","criacao de uma ciclovia, para seguranca de que...",oficina de violao
...,...,...,...,...,...,...,...,...
606,mamb,2,0,"melhora na qualidade e alcance da internet, ma...","areas para descanso para o intervalo, colchone...","almoco para todos, lanches diarios, compra de ...","criacao de uma ciclovia, para seguranca de que...",recuperacao e ocupacao do antigo cine januaria
607,mamb,2,0,"melhora na qualidade e alcance da internet, ma...","areas para descanso para o intervalo, microond...","almoco para todos, lanches diarios, poder paga...","criacao de uma ciclovia, para seguranca de que...",nenhuma opcao
608,mamb,2,0,"melhora na qualidade e alcance da internet, ma...","areas para descanso para o intervalo, colchone...","almoco para todos, lanches diarios, compra de ...","criacao de uma ciclovia, para seguranca de que...",recuperacao e ocupacao do antigo cine januaria
609,mamb,2,0,"melhora na qualidade e alcance da internet, ma...",areas para descanso para o intervalo,compra de ticktes on-line (sem precisar enfren...,"criacao de uma ciclovia, para seguranca de que...",oficina de violao


##### base para dashboard

In [30]:
df.drop(columns=[
    # 'atv_culturais',
    # 'hab_musicais',
    # 'esportes',
    # 'jogos',
    # 'lutas',
    # 'atv_culturais_a_fazer',
    # 'hab_musicais_desejadas',
    # 'esportes_desejados',
    # 'jogos_desejados',
    # 'lutas_desejadas',
    # Analisar como fazer a tratativa nessas colunas ao invés de remover
    # 'melhoria_estrutura',
    'novas_estruturas',
    # 'melhoria_alimentacao',
    'transporte',
    'projetos_a_participar',
], inplace=True)

df.to_csv('../data/data.csv', index=False)